In [ ]:
!pip install keras_vggface

In [ ]:
!pip install keras_applications

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##Import Statements


In [4]:
import numpy as np
from numpy import asarray
import pandas as pd
import os
from keras.optimizers import Adam
import pickle
import random
from PIL import Image, ImageOps
from collections import Counter
import glob
import itertools
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow
from tensorflow.keras import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Convolution2D, AveragePooling2D
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.layers import Input, Lambda
from keras.models import load_model, Model
from keras.preprocessing import image
from keras.utils import Sequence
from keras_vggface import utils
from keras_vggface.vggface import VGGFace
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score
import tensorflow as tf
from tensorflow.keras.layers import Dense

###Reading the all the possible Combinations

In [ ]:
photo_combinations = pd.read_csv("photo_combinations.csv",converters={"tuples":eval})
photo_combinations

In [6]:
photo_combinations = photo_combinations.sample(frac = 0.3547181055215420305483232475152)

###Train Test Split

In [7]:
def train_validate_test_split(df,seed= 2018, train_percent=.80, validation_percent = 0.125):
  train, test = train_test_split(df, train_size=train_percent, stratify=df['kinship'])
  train,validate = train_test_split(train, test_size=validation_percent, stratify=train['kinship'])
  return train, validate, test

In [8]:
train_df, validate_df, test_df = train_validate_test_split(photo_combinations)
train_df.reset_index(drop = True,inplace=True)
validate_df.reset_index(drop = True,inplace=True)
test_df.reset_index(drop = True,inplace=True)

In [9]:
def dual_image_reshape(img1_path, img2_path):
  img1 = asarray(ImageOps.grayscale(Image.open(img1_path)))
  img2 = asarray(ImageOps.grayscale(Image.open(img2_path)))
  return img1 - img2

In [10]:
def get_numpy_arr(df, directory):
  img_arr = []  #np.empty((224, 224), dtype='uint8')
  for i in range(len(df)):
    print(i)
    f_name1 = df.tuples[i][0].replace("\\","/" )
    path_1 = os.path.join(directory, f_name1)
    f_name2 = df.tuples[i][1].replace("\\","/" )
    path_2 = os.path.join(directory, f_name2)
    img_arr.append(dual_image_reshape(path_1, path_2))
    print('='*10)
  return np.array(img_arr)

In [ ]:
directory = 'drive/MyDrive/Project_Dataset'
X_train=get_numpy_arr(train_df, directory)
print("Train done")
print('='*50)

In [ ]:
X_valid=get_numpy_arr(validate_df, directory)
print("validation done")
print('='*50)

In [ ]:
X_test=get_numpy_arr(test_df, directory)
print("Test done")
print('='*50)

In [ ]:
y_train=train_df['kinship'].values
y_train=to_categorical(y_train, 2)
y_valid=validate_df['kinship'].values
y_valid=to_categorical(y_valid, 2)
y_actual = test_df['kinship'].values
y_actual=to_categorical(y_actual, 2)

In [ ]:
X_train = X_train.reshape(train_df.shape[0],224,224,1).astype('float32')
X_train /= 255

X_valid=X_valid.reshape( validate_df.shape[0],224,224,1).astype('float32')
X_valid /= 255

X_test = X_test.reshape(test_df.shape[0],224,224,1).astype('float32')
X_test /= 255

In [ ]:
# Layer 1 - Convolutional layer: To convert our images to inputs vectors
Seven_layer_model=Sequential()
Seven_layer_model.add(Conv2D(64, kernel_size=(3,3), activation='relu', input_shape=(224,224,1)))
Seven_layer_model.add(Dropout(0.25))
Seven_layer_model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
Seven_layer_model.add(Dropout(0.25))
Seven_layer_model.add(Conv2D(64, kernel_size=(3,3)))
Seven_layer_model.add(Dropout(0.25))
Seven_layer_model.add(AveragePooling2D(pool_size=(2,2)))
Seven_layer_model.add(BatchNormalization())

Seven_layer_model.add(Flatten())

# Layer 2 - Dense layer: ANN 
Seven_layer_model.add(Dense(128, activation='relu'))
Seven_layer_model.add(Dropout(0.32))
Seven_layer_model.add(BatchNormalization())
Seven_layer_model.add(Dense(64, activation='relu'))
Seven_layer_model.add(Dropout(0.3))
Seven_layer_model.add(BatchNormalization())

Seven_layer_model.add(Dense(32, activation='relu'))
Seven_layer_model.add(Dropout(0.2))
Seven_layer_model.add(BatchNormalization())
# Output Layer to 0/1
Seven_layer_model.add(Dense(2, activation='sigmoid'))

In [ ]:
Seven_layer_model.compile(
    loss='categorical_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy','AUC']
)

In [ ]:
Seven_layer_model.summary()

In [ ]:
Seven_layer_model.fit(
    X_train, 
    y_train, 
    batch_size=128, 
    epochs=30, 
    verbose=1, 
    validation_data=(X_valid, y_valid)
)

In [ ]:
Seven_layer_model.evaluate(X_test, y_actual)